In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sqlalchemy import create_engine

ImportError: cannot import name '_fit_context' from 'sklearn.base' (C:\Users\HP\anaconda3\lib\site-packages\sklearn\base.py)

In [53]:
project_dir = "C:\\Users\\HP\\Desktop\\major project\\"

In [55]:
os.chdir("C:\\Users\\HP\\Desktop\\major project")

In [56]:
messages = pd.read_csv("data/disaster_messages.csv")
categories = pd.read_csv("data/disaster_categories.csv")

In [57]:
df = pd.merge(messages, categories, on='id')

In [92]:
categories_expanded = categories['categories'].str.split(';', expand=True)
row = categories_expanded.iloc[0]
category_colnames = row.apply(lambda x: x.split('-')[0]).tolist()
categories_expanded.columns = category_colnames

In [91]:
#for column in categories_expanded:
   # categories_expanded[column] = categories_expanded[column].apply(lambda x: int(x.split('-')[1]))

In [132]:
def load_data():
    """Function to load database file
    
    Parameters
    -------
    none
    
    Returns
    -------
    Pandas DataFrame
    """
    #load data from database
    db_path= "C:\\Users\\HP\\Desktop\\major project\\data\\database.db"
    db_url = f"sqlite:///{db_path}"
    engine = create_engine(db_url)
    df= pd.read_sql_table('messages',engine)
    
    X= df['message']
    y= df.iloc[:,4:]
    return X,y

In [133]:
X, y = load_data()

In [134]:
def consolidate_classification_reports(y_test, y_pred):
    y_pred = pd.DataFrame(y_pred , columns = y_test.columns)
    
    feature_scores = pd.DataFrame()
    
    for col in y_test.columns:
        report_dict = classification_report(y_test[col],y_pred[col],output_dict = True)
        report_df = pd.DataFrame.from_dict(report_dict)
        report_df = report_df.iloc[:-1]
        report_df = pd.DataFrame(report_df.transpose().mean())
        report_df = report_df.transpose()
        feature_scores = pd.concat([feature_scores, report_df], ignore_index=True)

    return feature_scores 
        

In [135]:
def print_classification_results(y_test, y_pred):
    y_pred = pd.DataFrame(y_pred, columns = y_test.columns)
    for col in y_test.columns:
        print("Category feature : {}".format(col.capitalize()))
        print(classification_report(y_test[col],y_pred[col]))
        accuracy = (y_pred[col].values == y_test[col].values).mean().mean()*100
        print('Accuracy: {0:.1f} %\n'.format(accuracy))

In [136]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urplaceholder")
        
        
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
        
    return clean_tokens    

In [137]:
def build_model():
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf',TfidfTransformer()),
        ('clf',MultiOutputClassifier(RandomForestClassifier())),
        
    ])
    return pipeline

In [138]:
def build_new_model():
    pipeline=Pipeline([
        ('features',FeatureUnion([
            
            ('text_pipeline', Pipeline([
                ('vect',CountVectorizer(tokenizer=tokenize)),
                ('tfidf',TfidfTransformer())
            ]))
        ])),
        
        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])
    return pipeline

In [139]:
pipeline = build_model()

In [140]:
print(pipeline)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000002107EFA1DC0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])


In [141]:
X_train , X_test , y_train , y_test = train_test_split(X, y)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    pipeline.fit(X_train,y_train)

In [142]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [143]:
y_pred = pipeline.predict(X_test)


In [144]:
print_classification_results(y_test, y_pred)

Category feature : Related
              precision    recall  f1-score   support

         0.0       0.42      0.03      0.05      1526
         1.0       0.77      0.99      0.86      4937

    accuracy                           0.76      6463
   macro avg       0.59      0.51      0.46      6463
weighted avg       0.68      0.76      0.67      6463

Accuracy: 76.2 %

Category feature : Request
              precision    recall  f1-score   support

         0.0       0.83      1.00      0.91      5342
         1.0       0.56      0.02      0.05      1121

    accuracy                           0.83      6463
   macro avg       0.70      0.51      0.48      6463
weighted avg       0.78      0.83      0.76      6463

Accuracy: 82.7 %

Category feature : Offer
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      6434
         1.0       0.00      0.00      0.00        29

    accuracy                           1.00      6463
   macro avg 

C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classif

              precision    recall  f1-score   support

         0.0       0.97      1.00      0.98      6258
         1.0       0.00      0.00      0.00       205

    accuracy                           0.97      6463
   macro avg       0.48      0.50      0.49      6463
weighted avg       0.94      0.97      0.95      6463

Accuracy: 96.8 %

Category feature : Child_alone
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      6463

    accuracy                           1.00      6463
   macro avg       1.00      1.00      1.00      6463
weighted avg       1.00      1.00      1.00      6463

Accuracy: 100.0 %

Category feature : Water
              precision    recall  f1-score   support

         0.0       0.93      1.00      0.96      6031
         1.0       0.00      0.00      0.00       432

    accuracy                           0.93      6463
   macro avg       0.47      0.50      0.48      6463
weighted avg       0.87      0.93  

C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classif

              precision    recall  f1-score   support

         0.0       0.95      1.00      0.98      6150
         1.0       0.25      0.00      0.01       313

    accuracy                           0.95      6463
   macro avg       0.60      0.50      0.49      6463
weighted avg       0.92      0.95      0.93      6463

Accuracy: 95.1 %

Category feature : Other_aid
              precision    recall  f1-score   support

         0.0       0.86      0.99      0.92      5566
         1.0       0.10      0.00      0.01       897

    accuracy                           0.86      6463
   macro avg       0.48      0.50      0.46      6463
weighted avg       0.75      0.86      0.80      6463

Accuracy: 85.7 %

Category feature : Infrastructure_related
              precision    recall  f1-score   support

         0.0       0.93      1.00      0.97      6030
         1.0       0.50      0.01      0.01       433

    accuracy                           0.93      6463
   macro avg       0.

C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classif

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      6434
         1.0       0.00      0.00      0.00        29

    accuracy                           1.00      6463
   macro avg       0.50      0.50      0.50      6463
weighted avg       0.99      1.00      0.99      6463

Accuracy: 99.6 %

Category feature : Aid_centers
              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99      6377
         1.0       0.00      0.00      0.00        86

    accuracy                           0.99      6463
   macro avg       0.49      0.50      0.50      6463
weighted avg       0.97      0.99      0.98      6463

Accuracy: 98.7 %

Category feature : Other_infrastructure
              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98      6172
         1.0       0.60      0.01      0.02       291

    accuracy                           0.96      6463
   macro avg       0.

C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classif

In [145]:
scores = consolidate_classification_reports(y_test , y_pred)

C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classif

In [146]:
scores.head()

,precision,recall,f1-score
0,0.645005,0.609012,0.560111
1,0.739716,0.637038,0.602138
2,0.695966,0.698205,0.697083
3,0.555749,0.543591,0.494604
4,0.626378,0.666090,0.645382


In [155]:
scores

,precision,recall,f1-score
0,0.645005,0.609012,0.560111
1,0.739716,0.637038,0.602138
2,0.695966,0.698205,0.697083
3,0.555749,0.543591,0.494604
4,0.626378,0.666090,0.645382
5,0.654428,0.679133,0.666461
6,0.672857,0.687760,0.680194
7,0.682359,0.692031,0.687147
8,0.671654,0.687312,0.679357
9,1.000000,1.000000,1.000000


In [156]:
overall_accuracy =(y_pred == y_test).mean().mean()*100
print("overall accuracy :{0:1f}%".format(overall_accuracy))

overall accuracy :92.560644%


In [157]:
print("overall accuracy :{0:1f}%".format(overall_accuracy))

overall accuracy :92.560644%


In [158]:
scores['f1-score'].mean()

0.6684690287370031

In [159]:
print(y_pred)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [160]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', RandomForestClassifier())
])

pipeline.fit(X_train, y_train)

# Function to predict category for a given message
def predict_category(message):
    # Convert the message to a pandas DataFrame
    df = pd.DataFrame([message], columns=['message'])
    # Make predictions using the pipeline
    predictions = pipeline.predict(df['message'])
    return predictions


message = "the storm comes from delhi direction"

# Predict the category
predicted_category = predict_category(message)

# Print the predicted category
print(predicted_category)


[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [161]:
# Function to predict category for a given message
def predict_category(message):
    # Convert the message to a pandas DataFrame
    df = pd.DataFrame([message], columns=['message'])
    # Make predictions using the pipeline
    predictions = pipeline.predict(df['message'])
    return predictions


In [162]:
message = "the storm comes from delhi direction"


In [163]:
# Predict the category
predicted_category = predict_category(message)

# Print the predicted category
print(predicted_category)


[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [164]:
def predict_category(input_text):
    # Predict using the trained model
    predicted_labels = pipeline.predict([input_text])
    # Convert predicted labels back to category names using the original column names
    predicted_category_names = [category for category, value in zip(category_colnames, predicted_labels[0]) if value == 1]
    return predicted_category_names

In [165]:
input_text = "Is the Hurricane over or its not over"


In [166]:
predicted_category = predict_category(input_text)


In [167]:
print("Predicted Category",predicted_category)

Predicted Category ['related', 'aid_related', 'other_aid', 'weather_related', 'storm']


In [174]:
input_text2 = "we are asking for water , food"

In [175]:
predicted_category = predict_category(input_text2)

In [176]:
print("Predicted Category", predicted_category)

Predicted Category ['related', 'request', 'aid_related', 'direct_report']


In [177]:

def predict_category(message):
    # Convert the message to a pandas DataFrame
    df = pd.DataFrame([message], columns=['message'])
    # Make predictions using the pipeline
    predictions = pipeline.predict(df['message'])
    return predictions


In [178]:
message = "the storm comes from delhi direction"

In [179]:
predicted_category = predict_category(message)

In [180]:
predicted_category

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]])

In [181]:
filename= project_dir+ 'Classifier.model'

In [183]:
with open(filename,'wb') as f:
     pickle.dump(pipeline,f)